In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),transforms.ToPILImage(),transforms.Resize([50,50]),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [4]:
di = "data/train_images"

In [5]:
def load_split_train_test(datadir, test_size = .2):
    train_data = datasets.ImageFolder(datadir,       
                    transform=transform)
    test_data = datasets.ImageFolder(datadir,
                    transform=transform)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(test_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    return train_data, test_data, train_sampler, test_sampler
train_data, test_data, train_sampler, test_sampler = load_split_train_test(di, .2)

In [6]:
train_data[0][0].shape

torch.Size([3, 50, 50])

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [8]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [9]:
outputSize(3*32*32,3,1,1)

3072

In [10]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 25 * 25, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 5)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 25 * 25)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [11]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [12]:
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=8)
    return(train_loader)

In [13]:
get_train_loader(32)

In [14]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_data, batch_size=128, sampler=train_sampler, num_workers=8)

In [15]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [ ]:
CNN = SimpleCNN()

In [20]:
trainNet(CNN, batch_size=32, n_epochs=1, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 1
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 0.84 took: 28.54s
Epoch 1, 21% 	 train_loss: 0.83 took: 14.27s
Epoch 1, 32% 	 train_loss: 0.80 took: 12.27s
Epoch 1, 43% 	 train_loss: 0.76 took: 13.43s
Epoch 1, 54% 	 train_loss: 0.84 took: 26.84s
Epoch 1, 65% 	 train_loss: 0.79 took: 12.64s
Epoch 1, 76% 	 train_loss: 0.73 took: 13.97s
Epoch 1, 86% 	 train_loss: 0.81 took: 11.08s
Epoch 1, 97% 	 train_loss: 0.75 took: 18.27s


KeyboardInterrupt: 

In [ ]:
CNN.weights()